In [28]:
from mpi4py import MPI
import numpy as np
import random
import math
import ipyparallel as ipp
remote = ipp.Client()
print(remote.ids)
dview = remote[:]

[0, 1, 2, 3, 4, 5, 6, 7]


In [47]:
X = [random.randint(1, 50) for _ in range(40)]
Y = [random.randint(1, 50) for _ in range(40)]
# 分派data
dview.scatter('X', X)
dview.scatter('Y', Y)

<AsyncResult: scatter>

In [62]:
# 計算X變異數
@dview.remote(block=True)
def local_sum():
    return sum(X)

@dview.remote(block=True)
def local_var_sum_arg(mu):
    l_sum = 0
    for i in X:
        l_sum += (i - mu)**2
    return l_sum

l_sum = local_sum()
print('Partial sum', l_sum)
g_sum = sum(l_sum)
print('Global sum', g_sum)
gmu = g_sum / len(X)
print('Mean', gmu)
l_sum = local_var_sum_arg(gmu)
print('Partial variance sum', l_sum)
g_sum = sum(l_sum)
g_var = math.sqrt(g_sum / len(X))
print('SD = ', round(g_var,4))
#儲存X的標準差
X_sd = g_var

Partial sum [78, 131, 82, 179, 185, 171, 119, 100]
Global sum 1045
Mean 26.125
Partial variance sum [1809.078125, 1148.828125, 1558.078125, 1174.828125, 1651.328125, 862.828125, 237.828125, 1163.578125]
SD =  15.4971


In [57]:
# 計算Y變異數
@dview.remote(block=True)
def local_sum():
    return sum(Y)

@dview.remote(block=True)
def local_var_sum_arg(mu):
    l_sum = 0
    for i in Y:
        l_sum += (i - mu)**2
    return l_sum

l_sum = local_sum()
print('Partial sum', l_sum)
g_sum = sum(l_sum)
print('Global sum', g_sum)
gmu = g_sum / len(Y)
print('Mean', gmu)
l_sum = local_var_sum_arg(gmu)
print('Partial variance sum', l_sum)
g_sum = sum(l_sum)
g_var = math.sqrt(g_sum / len(Y))
print('SD = ', round(g_var,4))
#儲存Y的標準差
Y_sd = g_var

Partial sum [135, 93, 101, 163, 78, 86, 120, 145]
Global sum 921
Mean 23.025
Partial variance sum [1839.0031250000004, 1227.1031249999999, 1234.703125, 1175.603125, 980.8531249999999, 752.453125, 998.753125, 428.5031250000001]
SD =  14.6944


In [58]:
@dview.remote(block=True)
def local_cov_sum_arg(mu_X, mu_Y):
    l_sum = 0
    for i in range(len(X)):
        l_sum += (X[i] - mu_X) * (Y[i] - mu_Y)
    return l_sum

In [59]:
gmu_X = sum(X) / len(X)
gmu_Y = sum(Y) / len(Y)

In [60]:
l_sum = local_cov_sum_arg(gmu_X, gmu_Y)
print('Partial covariance sum', l_sum)
g_sum = sum(l_sum)
g_cov = g_sum / len(X)
print('Covariance = ', round(g_cov,4))
#儲存X&Y的共變數
XY_cov = g_cov

Partial covariance sum [-1132.184375, 578.740625, 1179.965625, 905.7906250000001, -11.734374999999886, -192.38437499999992, 280.66562500000003, -76.98437500000009]
Covariance =  38.2969


In [61]:
#計算相關係數
r = XY_cov/(X_sd*Y_sd)
print("correlation coefficient:",round(r,4))

correlation coefficient: 0.1682


In [55]:
print(X,Y)

[5, 9, 6, 47, 11, 44, 2, 36, 16, 33, 2, 3, 13, 42, 22, 16, 28, 45, 45, 45, 38, 9, 46, 42, 50, 32, 40, 36, 16, 47, 32, 16, 21, 19, 31, 9, 37, 11, 37, 6] [50, 4, 42, 6, 33, 9, 1, 28, 12, 43, 6, 7, 10, 41, 37, 13, 40, 46, 39, 25, 6, 10, 11, 12, 39, 3, 17, 8, 26, 32, 44, 2, 19, 33, 22, 21, 38, 24, 25, 37]
